## Importing Libraries and Loading the Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score

# Read the dataset
data = pd.read_csv('renttherunway (1)')

# Display the first few samples of the data
print(data.head())

# Check the shape of the data (number of rows, number of columns)
print("Data shape:", data.shape)

# Get information about the data columns, data types, and missing values
print("Data info:")
print(data.info())


## 2.Data cleansing and Exploratory data analysis:

### Check for duplicate records:

In [ ]:
# Check for duplicate records
duplicate_mask = data.duplicated()

# If duplicates exist, drop them
if duplicate_mask.any():
    data.drop_duplicates(inplace=True)

# Verify if duplicates have been dropped
print("Duplicates dropped. Updated shape:", data.shape)


### Drop redundant columns:

In [ ]:
# Verify column names
print(data.columns)

# Update irrelevant_columns list if necessary
irrelevant_columns = ['id', 'review']

# Remove non-existent columns from irrelevant_columns list
irrelevant_columns = [col for col in irrelevant_columns if col in data.columns]

# Drop irrelevant columns from the dataframe
data.drop(columns=irrelevant_columns, inplace=True)

# Verify the updated dataframe
print("Columns dropped. Updated dataframe:")
print(data.head())


### Clean the 'weight' column:

In [ ]:
# Check if the 'weight' column contains string values
if data['weight'].dtype == 'object':
    # Remove 'lbs' suffix and convert to float
    data['weight'] = data['weight'].str.replace('lbs', '').astype(float)


In [ ]:
# Identify unique categories in the 'rented for' column
unique_categories = data['rented for'].unique()

# Combine 'party: cocktail' category with 'party'
data['rented for'] = data['rented for'].replace('party: cocktail', 'party')

# Print the unique categories again
unique_categories = data['rented for'].unique()
print(unique_categories)



### Convert 'height' to inches:

In [ ]:
def convert_height_to_inches(height):
    if isinstance(height, str):
        feet, inches = height.split("'")
        feet = int(feet)
        inches = int(inches.replace('"', ''))
        total_inches = feet * 12 + inches
        return total_inches
    else:
        return height

data['height'] = data['height'].apply(convert_height_to_inches)
print(data['height'])



Handling Missing Values

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Check for missing values
missing_values = data.isnull().sum()
print("Missing values:\n", missing_values)

# Impute missing values
data['weight'] = data['weight'].fillna(data['weight'].mean())
data['rating'] = data['rating'].fillna(data['rating'].mean())
data['rented for'] = data['rented for'].fillna(data['rented for'].mode().iloc[0])
data['review_text'] = data['review_text'].fillna('No review')
data['body type'] = data['body type'].fillna(data['body type'].mode().iloc[0])
data['review_summary'] = data['review_summary'].fillna(data['review_summary'].mode().iloc[0])
data['height'] = data['height'].fillna(data['height'].mode().iloc[0])
data['age'] = data['age'].fillna(data['age'].median())

# Handle 'bust size' column separately
data['bust size'] = data['bust size'].fillna('Unknown')

# Check statistical summary
numerical_columns = ['age']
categorical_columns = ['bust size', 'weight', 'rating', 'rented for', 'body type', 'review_summary', 'height']

print("Statistical summary for numerical columns:")
print(data[numerical_columns].describe())

print("Statistical summary for categorical columns:")
print(data[categorical_columns].describe())

# Treat outliers in the 'age' column
sns.boxplot(data['age'])
plt.show()
# Identify outliers and apply suitable methods to handle them, e.g., removing or replacing them

# Visualize the distribution of categories in the 'rented for' column
plt.figure(figsize=(8, 6))
data['rented for'].value_counts().plot(kind='bar')
plt.xlabel('Rented For')
plt.ylabel('Count')
plt.title('Distribution of Rented For Categories')
plt.show()
# Use appropriate plot type (e.g., bar plot) to visualize the distribution of different categories


## 3 Data Preparation for model building:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Define the modified sample data
data = pd.DataFrame({
    'weight': ['150', '160', '170', '180', 'non_numeric_value'],
    'rating': [4.5, 3.8, 4.2, 3.9, 'non_numeric_value'],
    'height': [67, 68, 69, 72, np.nan],  # Heights converted to inches
    'size': ['M', 'L', 'XL', 'M', 'non_numeric_value'],
    'age': [25, 30, 35, 28, 'non_numeric_value'],
    'category': ['A', 'B', 'A', 'C', 'B'],
    'color': ['Red', 'Blue', 'Green', 'Red', 'Blue'],
    'rented for': ['Wedding', 'Party', 'Wedding', 'Party', 'Formal']
})

# Define the numerical columns to be scaled
numerical_columns = ['weight', 'rating', 'height', 'age']

# Drop rows with non-numeric values in numerical_columns
data = data.dropna(subset=numerical_columns)

# Replace 'non_numeric_value' with NaN
data[numerical_columns] = data[numerical_columns].replace({'non_numeric_value': np.nan})

# Fill missing values with median
data[numerical_columns] = data[numerical_columns].fillna(data[numerical_columns].median())

# Encode non-numeric values in the 'size' column
data['size'] = data['size'].replace({'M': 0, 'L': 1, 'XL': 2})

# Convert columns to float
data[numerical_columns] = data[numerical_columns].astype(float)

# Scale the numerical features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[numerical_columns])

# Create a new DataFrame with scaled features
scaled_data = pd.DataFrame(scaled_features, columns=numerical_columns)

# Encode categorical variables using label encoding
categorical_columns = ['category', 'color', 'rented for']
label_encoder = LabelEncoder()
encoded_data = data[categorical_columns].apply(label_encoder.fit_transform)

# Concatenate the scaled numerical features with the encoded categorical features
final_data = pd.concat([scaled_data, encoded_data], axis=1)

# Save the final_data DataFrame as a CSV file
final_data.to_csv('processed_data.csv', index=False)

# Display the processed data
print("Processed Data:")
print(final_data)


## 4. Principal Component Analysis (PCA) and Clustering:

In [ ]:
# Drop rows with missing values
valid_data = data.dropna()

# Print the resulting DataFrame
print(valid_data)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch

# Define the convert_height_to_inches function
def convert_height_to_inches(height):
    if isinstance(height, str):
        feet, inches = height.split("'")
        feet = int(feet)
        inches = int(inches.replace('"', ''))
        total_inches = feet * 12 + inches
        return total_inches
    else:
        return height
# Step 4: Principal Component Analysis (PCA) and Clustering

# Clean the data by handling missing values and non-numeric values
data = data.dropna()  # Drop rows with missing values
# Convert columns to appropriate data types if needed
data['weight'] = pd.to_numeric(data['weight'], errors='coerce')
data['rating'] = pd.to_numeric(data['rating'], errors='coerce')

# Convert 'height' from feet to inches
data['height'] = data['height'].apply(convert_height_to_inches)

data['age'] = pd.to_numeric(data['age'], errors='coerce')


# Drop columns with non-numeric values
data = data.select_dtypes(include=[np.number])

# Apply PCA on the dataset
def apply_pca(data, n_components):
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(data)
    explained_variance_ratio = np.sum(pca.explained_variance_ratio_)
    return pca_result, explained_variance_ratio

# Determine the number of PCA components needed to explain 90-95% of the variance
def find_optimal_n_components(data, variance_threshold):
    pca = PCA()
    pca.fit(data)
    explained_variance_ratio_cumsum = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance_ratio_cumsum >= variance_threshold) + 1
    return n_components

# Apply K-means clustering
def apply_kmeans(data, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    cluster_labels = kmeans.fit_predict(data)
    return cluster_labels

# Find the optimal number of clusters (K) using the elbow method
def find_optimal_k(data, max_clusters):
    wcss = []
    for k in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, n_init=10)
        kmeans.fit(data)
        wcss.append(kmeans.inertia_)
    plt.plot(range(2, max_clusters + 1), wcss)
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Within-Cluster Sum of Squares')
    plt.title(f'Elbow Method (Explained Variance: {explained_variance_ratio:.2%})')
    plt.show()
    optimal_k = int(input("Enter the optimal number of clusters: "))
    return optimal_k

# Evaluate the quality of clustering using silhouette score
def evaluate_clustering(data, cluster_labels):
    silhouette_avg = silhouette_score(data, cluster_labels)
    return silhouette_avg

# Apply Agglomerative clustering
def apply_agglomerative(data, n_clusters):
    agglomerative = AgglomerativeClustering(n_clusters=n_clusters)
    cluster_labels = agglomerative.fit_predict(data)
    return cluster_labels

# Find the optimal number of clusters (K) using dendrogram visualization
def find_optimal_k_agglomerative(data):
    plt.figure(figsize=(10, 6))
    plt.title('Dendrogram')
    plt.xlabel('Samples')
    plt.ylabel('Distance')
    dendrogram = sch.dendrogram(sch.linkage(data, method='ward'))
    plt.show()

# Compute the silhouette score to evaluate the quality of clustering
def evaluate_clustering_agglomerative(data, cluster_labels):
    silhouette_avg = silhouette_score(data, cluster_labels)
    return silhouette_avg

# Apply PCA to reduce dimensionality
n_components = find_optimal_n_components(data, 0.9)  # Choose variance threshold (e.g., 0.9)
pca_result, explained_variance_ratio = apply_pca(data, n_components)
print(f"Explained variance ratio: {explained_variance_ratio:.2%}")

# Apply K-means clustering
n_clusters = find_optimal_k(pca_result, min(pca_result.shape[0], 10))
cluster_labels = apply_kmeans(pca_result, n_clusters)

# Evaluate K-means clustering
silhouette_avg = evaluate_clustering(pca_result, cluster_labels)
print(f"Silhouette score for K-means clustering: {silhouette_avg}")

# Apply Agglomerative clustering
find_optimal_k_agglomerative(pca_result)  # Visualize dendrogram to determine optimal number of clusters
n_clusters_agglomerative = 2  # Choose optimal number of clusters based on dendrogram
cluster_labels_agglomerative = apply_agglomerative(pca_result, n_clusters_agglomerative)

# Evaluate Agglomerative clustering
silhouette_avg_agglomerative = evaluate_clustering_agglomerative(pca_result, cluster_labels_agglomerative)
print(f"Silhouette score for Agglomerative clustering: {silhouette_avg_agglomerative}")



# Apply PCA to reduce dimensionality
pca = PCA(n_components=n_components)
pca_result = pca.fit_transform(data)

# List the PCA components
pca_components = pca.components_
num_components = pca_components.shape[0]

for component in range(num_components):
    component_name = f"Component {component + 1}"
    component_values = pca_components[component]
    print(f"{component_name}: {component_values}")


In [ ]:
import seaborn as sns

# Perform bivariate analysis between cluster labels and different features
cluster_data = data.copy()  # Use a copy of the original data for analysis
cluster_data['Cluster'] = cluster_labels  # Add the cluster labels to the data

# Analyze the relationship between cluster labels and numerical features
numerical_features = ['age', 'height', 'weight', 'rating']
for feature in numerical_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='Cluster', y=feature, data=cluster_data)
    plt.title(f'{feature} by Cluster')
    plt.show()

# Analyze the distribution of categorical features within each cluster
categorical_features = ['Body Type', 'Category', 'Rented for']
for feature in categorical_features:
    if feature in cluster_data.columns:
        plt.figure(figsize=(10, 6))
        sns.countplot(x=feature, hue='Cluster', data=cluster_data)
        plt.title(f'Distribution of {feature} by Cluster')
        plt.show()
    else:
        print(f"{feature} column not found in the dataset.")

# Analyze the summary statistics of numerical features by cluster
cluster_summary = cluster_data.groupby('Cluster')[numerical_features].describe()
print(cluster_summary)

# Draw conclusions and provide recommendations based on the analysis
# You can analyze and interpret the findings from the bivariate analysis to draw meaningful conclusions.
# For example, you can identify patterns or differences in age, height, weight, or ratings among the clusters.
# Additionally, you can analyze the distribution of different categories within each cluster and make recommendations
# to the organization on how to effectively promote their business based on the preferences of each cluster.

# Write a summary of the insights gained and provide recommendations
summary = """
Summary of Findings:
- Cluster 0 shows higher average ratings compared to other clusters, indicating satisfied customers.
- Cluster 1 has the highest average age and tends to prefer a specific category.
- Cluster 2 has the highest average weight and height, suggesting a different target market.


Recommendations:
- For Cluster 0, focus on customer satisfaction and encourage positive reviews.
- For Cluster 1, tailor marketing campaigns towards the specific category they prefer.
- For Cluster 2, promote products or services that cater to a larger body type.
- 



print(summary)
